In [2]:
#game
from collections import namedtuple
from dataclasses import dataclass
from io import StringIO
from typing import NamedTuple

import numpy as np


class checkers:
    def __init__(self, height=8, width =8):
        self.board = np.zeros((8,8),dtype=np.int8)
        self.height = height
        self.width = width
        self.board[0:3:2, 1:8:2] = 1
        self.board[1, 0:8:2] = 1
        self.board[6:8:2, 1:8:2] = 2
        self.board[5:8:2, 0:8:2] = 2
        self.winner = 0

    @dataclass
    class Movement:
        start:NamedTuple("start",[('row',int), ('col',int)])
        end:NamedTuple("end",[('row',int), ('col',int)])
        jump:NamedTuple("jump",[('row',int), ('col',int)]) = None
        subsequent: list = None
        
        def __toChar(self,position):
            if position == None: return ""
            retval = chr(ord('`')+position[0]+1), str(position[1])
            return retval

        def __str__(self):
            val = ''.join(self.__toChar(self.start)) +'->'+''.join(self.__toChar(self.end))+('∩'+''.join(self.__toChar(self.jump)) if self.jump else '')
            return val
        
        def __repr__(self):
            return self.__str__()
        
    def _posToStr(self,position):
        if position == None: return ""
        return chr(ord('`')+position[0]+1) + str(position[1])
    
    def _strToPos(self,position):
        if position == None: return ""
        return int(ord(position[0])-ord('a')), int(position[1])

    def _valid(self,position):
        return self.__valid(position)
        
    def __valid(self,position):
        if position == None: return False
        r,c = position
        return r >= 0 and r < self.height and c >= 0 and c < self.width

    def _jumps(self,position, player = None, allowBackJump = True, exclude_from = (-1,-1)):
        if type(position) == type('str'): position = self._strToPos(position)
        jumps = []
    
        r,c = position
        if (player:= (player or self.board[position])) == 0:  return []
        opponent = 3^(player&3)
        if player&4 == 4: allowBackJump = True
        directions = []
        if player   & 4 == 4:  directions =  [(1,1),(2,2)], [(1,-1),(2,-2)], [(-1,1),(-2,2)], [(-1,-1),(-2,-2)]
        elif player & 1 == 1:  directions =  [(1,1),(2,2)], [(1,-1),(2,-2)]
        elif player & 2 == 2:  directions =  [(-1,1),(-2,2)], [(-1,-1),(-2,-2)] 
        else: return jumps
        
        for neighbor,drop in directions:
            n = (r+neighbor[0],c+neighbor[1])
            d = (r+drop[0],c+drop[1])
            if n == exclude_from or d==exclude_from: continue
            if(not self.__valid(n) or not self.__valid(d)): continue

            if (self.board[n]&3)==opponent and self.board[d] == 0:
                jump_move = self.Movement(position,d,n) 
                jumps.append(jump_move)
                p = (player&3|4) if allowBackJump else player
                multi_jumps = self._jumps(d, player = p, allowBackJump=allowBackJump, exclude_from = position)
                jump_move.subsequent = multi_jumps
        return jumps
         
    def _moves(self,position) -> list:
        if type(position) == type('str'): position = self._strToPos(position)
        row, col = position
        if (player:= self.board[position]) == 0:  return []
        directions = []
        if   player&4==4: directions = [(1,-1),(1,1),(-1,-1),(-1,1)] 
        elif player&2==2: directions = [(-1,-1),(-1,1)]
        elif player&1==1: directions = [(1,-1),(1,1)]

        for dr,dc in directions:
           if self.__valid((row+dr,col+dc)) and self.board[row+dr,col+dc] == 0: yield self.Movement(position,(row+dr,col+dc))
        
    def move(self,move:Movement):
        x1,y1 = move.start
        x2,y2 = move.end
        jx,jy = -1,-1 or move.jump
        if(move.jump): 
            self.jump(move)
            p = self.board[x2,y2]
        else:
            p = self.board[x1,y1]
            self.board[x1,y1] = 0
            self.board[x2,y2] = p

        if p&3 == 1 and x2 == self.height-1: self.board[x2,y2] = 5
        if p&3 == 2 and x2 == 0: self.board[x2,y2] = 6
        return move
    
    def jump(self,move:Movement):
        x1,y1 = move.start
        x2,y2 = move.end
        jx,jy = move.jump
        p = self.board[x1,y1]
        self.board[x1,y1] = 0
        self.board[x2,y2] = p
        self.board[jx,jy] = 0
        return move

    def __str__(self):
        # https://stackoverflow.com/questions/4842424/list-of-ansi-color-escape-sequences
        bold =  '\033[1m'
        borderColor = bold+'\033[90m'
        brightBorderColor = bold+'\033[37m'
        end_color = '\033[0m'
        underline = '\033[4m'
        red = bold+'\033[91m'
        green = bold+'\033[92m'
        out = StringIO()
        print('    ' + underline,end="",file=out)
        for i in range(self.board.shape[1]):
            print(f"   {i}  ",end="",file=out)
        print(' '+ end_color,file=out)
        for row in range(self.board.shape[0]):
            print("|",end="",file=out)
            print(" "*3,end="",file=out)
            print(f"{borderColor}+-----"*8, end=f"+\n",file=out)
            print(end_color + f"|{  chr(ord('`')+row+1)  }  "+ borderColor,end="",file=out)
            for col in range(self.board.shape[1]):
                color = red if self.board[row,col]&3&1 == 1 else green if self.board[row,col]&3&2 == 2 else borderColor
                char = 'K' if self.board[row,col] & 4 == 4 else '@' if self.board[row,col] != 0 else ' '
                print(f"| {color} {char}  {borderColor}",end="",file=out)
            print(f"|{end_color}",file=out)
                
        print("|",end="",file=out)
        print(" "*3,end="",file=out)
        print(f"{borderColor}+-----"*8, end=f"+{end_color}\n",file=out)
        return out.getvalue()
    
    def __repr__(self) -> str:
        return self.__str__()  


In [2]:
#move tree
from treelib import Tree
from treelib import Node as nd

def treeFrom(position, movements:list[checkers.Movement]):
    def r_build_tree(t:Tree, n:nd):
        if n.data == None: return
        for e in n.data:
            _n = t.create_node(str(e),parent=n, data = e.subsequent)
            r_build_tree(t,_n)
            
    if movements == None or len(movements) == 0: return None

    t=Tree() 
    _node = t.create_node(position,data = movements)
    r_build_tree(t,_node)
    return t

In [3]:
game = checkers()
game.move(checkers.Movement((2,1),(3,4)))
game.move(checkers.Movement((5,4),(4,3)))
game.move(checkers.Movement((5,2),(4,1)))
game.move(checkers.Movement((7,0),(3,2)))

mv = list(game._moves('d2'))
mv.extend(list(game._jumps('d2',allowBackJump=False)))

t = treeFrom('d2',mv)
if(t): t.show()


print(game)

d2
└── d2->c1

       0     1     2     3     4     5     6     7   
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|a  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|b  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|c  |     |     |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|d  |     |     |  @  |     |  @  |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|e  |     |  @  |     |  @  |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|f  |  @  |     |     |     |     |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|g  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|h  |     |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+



In [4]:
#simulate Game
#just take random moves for each player
import random
import IPython.display as display
game = checkers()



while True:
    redMoves = []
    redPlayers = list(zip(*np.where(game.board&3&1 == 1)))
    for p in redPlayers:
        redMoves.extend(list(game._moves(p)))
        redMoves.extend(list(game._jumps(p,allowBackJump=False)))
    if(len(redMoves) == 0): break
    i = input()
    if i == 'q': break
    display.clear_output(False)
    print('red')
    rm = random.choice(redMoves)
    print(list(map(str,redMoves)))
    game.move(rm)
    print(rm)
    while rm.subsequent != None and len(rm.subsequent) > 0:
        rm = random.choice(rm.subsequent)
        game.move(rm)
   
    print(game)

    greenMoves = []
    greenJumps = []

    greenPlayers = list(zip(*np.where(game.board&3&2 == 2)))
    for p in greenPlayers:
        greenMoves.extend(list(game._moves(p)))
        greenJumps.extend(list(game._jumps(p,allowBackJump=False)))

    if(len(greenMoves) == 0 and len(greenJumps)==0): 
        print(list(game._moves((5,0))))
        break

    i = input()
    if i == 'q': break
    display.clear_output(False)
    print('green')
    print(list(map(str,greenMoves)))
    
    if len(greenJumps) > 0:
        gm = random.choice(greenJumps)
    else:
        gm = random.choice(greenMoves)
 
    game.move(gm)
    print(gm)
    while gm.subsequent != None and len(gm.subsequent) > 0:
        gm = random.choice(gm.subsequent)
        game.move(gm)
        print(gm)
    
    
    print(game)



red
['c1->d0', 'c1->d2', 'c3->d2', 'c3->d4', 'c5->d4', 'c5->d6', 'c7->d6']
c1->d0
       0     1     2     3     4     5     6     7   
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|a  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|b  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|c  |     |     |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|d  |  @  |     |     |     |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|e  |     |     |     |     |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|f  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|g  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|h  |  @  |     |  @  |     |  @  |     |  @  |     |


In [31]:
#lets see how many simulations we can complete 
import time
import random
from typing import Callable
import concurrent.futures

def simulate(startPlayer:int=1, allowBackjump:bool=False, requireJump=True, maxMoves = 200)->int:
    '''
        Returns the Winner
    '''
    game = checkers()
    player = startPlayer
    m = maxMoves

    moves_1 = []
    moves_2 = []

    while m >= 0:
        moves = []
        jumps = []
        pieces = list(zip(*np.where(game.board&3&player == player)))
        
        for p in pieces:
            moves.extend(list(game._moves(p)))
            jumps.extend(list(game._jumps(p,allowBackJump=allowBackjump)))
            if not requireJump:
                moves.extend(jumps)
        if len(moves) == 0: break

        rm:checkers.Movement = None

        if requireJump and len(jumps) > 0:
            rm = random.choice(jumps) 
        else:
            rm = random.choice(moves)
        
        rm = game.move(rm)
        if player == 1: moves_1.append(rm)
        if player == 2: moves_2.append(rm)

        while rm.subsequent != None and len(rm.subsequent) > 0:
            rm = random.choice(rm.subsequent)
            game.move(rm)
            if player == 1: moves_1.append(rm)
            if player == 2: moves_2.append(rm)


        player = 3^(player&3)
        m -= 1
        
    return 0 if moves==0 else 3^(player&3),moves_1, moves_2


def runSimulations(count:int=1, callback:Callable[[int,int,list,list],None] = None):

    red_wins = 0
    green_wins = 0
    start = time.time()

    def __runsim(i):
        s = time.time()
        winner, moves1, moves2 = simulate()
        e = time.time()        
        if callback : callback(i, (e-s),winner,moves1,moves2)
        return (i,winner,moves1,moves2)

    with concurrent.futures.ThreadPoolExecutor(32) as executor:
        winners = list(executor.map(__runsim,range(count)))

    end = time.time()
    
    return (end-start), winners

In [ ]:
#mcst
#   Select
#   Expand
#   Simulate
#   Back propigate
import sys


sys.setrecursionlimit(10**6)
bold =  '\033[1m'
end_color = '\033[0m'
red = bold+'\033[91m'
green = bold+'\033[92m'
red_wins = 0
green_wins = 0

def cb(i, time, winner, moves1,moves2):
    print(f"Game {i}: Winner is player {f'{bold}{red}Red{end_color}' if winner == 1 else f'{bold}{green}Green{end_color}' if winner == 2 else f'{bold}Draw{end_color}'} in {time:.2f}s")
    print(f'\tTotal Moves Red:{len(moves1)}, Green:{len(moves2)}')

total_games = 1000
print(f'Running {total_games} games')
# totalRunTime, winners= runSimulations(total_games)
totalRunTime =runSimulations(total_games)
# red_wins = sum([1 for r in winners if r[1] == 1])
# green_wins = sum([1 for r in winners if r[1] == 2])

# print(f'{bold}{red}Red{end_color} wins {bold}{red}{red_wins}{end_color} times, {bold}{green}Green{end_color} wins {bold}{green}{green_wins}{end_color} times')
print(f'Total run time {totalRunTime}s')

#### New algorithim for checking for moves
* should change complexity to O(n) to O(1)

-LET T be the target cell  
-LET O be the orginating cell  
-LET J<sub>1..n</sub> be any jumped cells  
- Before Moving
-  - for all pieces P in adjacent cells not in T
-  - - Re-calculate Possible Moves
-  
-  Execute Move
-  
-  for each J<sub>n</sub> 
-  - for all pieces P adjacent to J<sub>n</sub> not previously calculated
-  - re-calculate possible moves
-  for all pieces P with T as a target or pieces adjacent to T
-  - re-cacluate piossible Moves 

In [13]:
from collections import namedtuple
from __future__ import annotations


Point = namedtuple("Point", "row col")



class Mvt:
    def __init__(self, start:Point, end:Point, jump:Point=None, subsequent:List[Mvt]=None):
        self.start = Point(*start)
        self.end = Point(*end)
        self.jump = Point(*jump) if jump else None
        self.subsequent = subsequent



    def __toChar(self,position):
        if position == None: return ""
        retval = chr(ord('`')+position[0]+1), str(position[1])
        return retval

    def __str__(self):
        val = ''.join(self.__toChar(self.start)) +'->'+''.join(self.__toChar(self.end))+('∩'+''.join(self.__toChar(self.jump)) if self.jump else '')
        return val
    
    def __repr__(self):
        return self.__str__()

    def __eq__(self,other):
        return hash(self) == hash(other)
    
    def __hash__(self):
        # return hash((hash(self.start),hash(self.end),hash(self.jump),hash(self.subsequent)))
        return hash((hash(self.start),hash(self.end)))

def adjacent(game, position) -> set:
    row,col = position
    retval = [] 
    for dr,dc in [(1,-1),(1,1),(-1,-1),(-1,1)]:
        if game._valid((row+dr,col+dc)): yield (row+dr,col+dc) 

    return

def moves(game,position) -> list:
    if type(position) == type('str'): position = self._strToPos(position)
    row, col = position
    if (player:= game.board[position]) == 0:  return []
    directions = []
    if   player&4==4: directions = [(1,-1),(1,1),(-1,-1),(-1,1)] 
    elif player&2==2: directions = [(-1,-1),(-1,1)]
    elif player&1==1: directions = [(1,-1),(1,1)]

    for dr,dc in directions:
        if game._valid((row+dr,col+dc)) and game.board[row+dr,col+dc] == 0: 
            yield Mvt(position,(row+dr,col+dc))

def GetAllMoves(game)-> set:
    pieces = list(zip(*np.where(game.board != 0)))
    retval = set()
    for p in pieces:
        retval.update(set(moves(game,p)))
    return retval

def move(game, mvt, available_moves):
    #pick up and move the piece
    origin = mvt.start
    target = mvt.end

    p = game.board[origin]
    game.board[origin] = 0

    available_moves = available_moves - {x for x in available_moves if x.start == Point(*origin)}

    #recalculate the moves for all adjacent pieces
    for a in adjacent(game,origin):
        mm = set(moves(game,a))
        available_moves = available_moves.union(mm)

    #drop the piece in the target
    game.board[target] = p

    adjacent_targets = list(adjacent(game,target))

    affected = {x for x in available_moves if x.end == target or x.start in adjacent_targets}
    available_moves = available_moves - affected

    for a in affected:
        available_moves = available_moves.union(set(moves(game,a.start)))

    available_moves = available_moves.union(set(moves(game,target)))
    return available_moves



In [17]:
import random
game = checkers()
allMoves = GetAllMoves(game)


origin = game._strToPos('c3')
target = game._strToPos('d2')
allMoves = move(game,Mvt(origin,target),allMoves)

print(allMoves)
print(mvs)
print()

playerMoves = lambda b,m,p: {x for x in m if b[x.start]&3&p == p}
print(f"Player 1 moves: {playerMoves(game.board,mvs,1)}")
print(f"Player 2 moves: {playerMoves(game.board,mvs,2)}")


print(game)




{f4->e5, c1->d2, c3->d4, f4->e3, c1->d0, f2->e1, f6->e5, c7->d6, f0->e1, c5->d4, f6->e7, c3->d2, c5->d6, f2->e3}
{f4->e5, f4->e3, b4->c3, f2->e1, f6->e5, c1->d0, c7->d6, f0->e1, c5->d4, f6->e7, d2->e3, d2->e1, c5->d6, f2->e3, b2->c3}

Player 1 moves: {d2->e3, b4->c3, c1->d0, c7->d6, c5->d4, c5->d6, d2->e1, b2->c3}
Player 2 moves: {f4->e5, f4->e3, f6->e5, f2->e1, f0->e1, f2->e3, f6->e7}
       0     1     2     3     4     5     6     7   
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|a  |     |  @  |     |  @  |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|b  |  @  |     |  @  |     |  @  |     |  @  |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|c  |     |  @  |     |     |     |  @  |     |  @  |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|d  |     |     |  @  |     |     |     |     |     |
|   +-----+-----+-----+-----+-----+-----+-----+-----+
|e  |     |     |     |     |     |     |     |     |
|   +-----+-----+